# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [176]:
# Load the libraries as required.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [177]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [178]:
X = fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']]
y = fires_dt['area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
numerical_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi','temp', 'rh', 'wind', 'rain', 'area']
categorical_features = ['month', 'day']

In [196]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore')
preproc1 = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_features),
        ('cat', encoder, categorical_features)
    ]
)

X = fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain','area']]  # Include both numerical and categorical features
X_preproc1 = preproc1.fit_transform(X)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [207]:

from sklearn.preprocessing import PowerTransformer, OneHotEncoder

non_linear_features =['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi','temp', 'rh', 'wind', 'rain']
power_transformer = PowerTransformer()
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore')
preproc2 = ColumnTransformer(
    transformers=[
        ('num_non_linear', power_transformer, non_linear_features),  
        ('num_scaled', scaler, [feature for feature in numerical_features if feature not in non_linear_features]),  
        ('cat', encoder, categorical_features)  
    ]
)
X = fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain','area']]
X_preproc2 = preproc2.fit_transform(X)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [182]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.pipeline import Pipeline

In [183]:
# Pipeline A = preproc1 + baseline
PipelineA = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', LinearRegression())
])

In [184]:
# Pipeline B = preproc2 + baseline
PipelineB = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', LinearRegression())
])

In [185]:
# Pipeline C = preproc1 + advanced model
PipelineC = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor())
])

In [186]:
# Pipeline D = preproc2 + advanced model
PipelineD = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor())
])
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [187]:
baseline_param_grid = {
    'regressor__fit_intercept': [True, False],
}

In [188]:
advanced_param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5]
}


In [189]:
baseline_grid_searchA = GridSearchCV(PipelineA, baseline_param_grid, cv=5, scoring='neg_mean_squared_error')
baseline_grid_searchA.fit(X, y)

# Evaluate Baseline Regressor
baseline_cv_scoresA = cross_val_score(baseline_grid_searchA.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
print("Baseline Regressor CV MSE:", -baseline_cv_scoresA.mean())

Baseline Regressor CV MSE: 1.3754019324324584e-25


In [190]:
baseline_grid_searchB = GridSearchCV(PipelineB, baseline_param_grid, cv=5, scoring='neg_mean_squared_error')
baseline_grid_searchB.fit(X, y)

# Evaluate Baseline Regressor
baseline_cv_scoresB = cross_val_score(baseline_grid_searchB.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
print("Baseline Regressor CV MSE:", -baseline_cv_scoresB.mean())

Baseline Regressor CV MSE: 4109.895747339568


In [191]:
advanced_grid_searchC = GridSearchCV(PipelineC, advanced_param_grid, cv=5, scoring='neg_mean_squared_error')
advanced_grid_searchC.fit(X, y)

# Evaluate Advanced Regressor
advanced_cv_scoresC = cross_val_score(advanced_grid_searchC.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
print("Advanced Regressor CV MSE:", -advanced_cv_scoresC.mean())


Advanced Regressor CV MSE: 628.71516417869


In [192]:
advanced_grid_searchD = GridSearchCV(PipelineD, advanced_param_grid, cv=5, scoring='neg_mean_squared_error')
advanced_grid_searchD.fit(X, y)

# Evaluate Advanced Regressor
advanced_cv_scoresD = cross_val_score(advanced_grid_searchD.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
print("Advanced Regressor CV MSE:", -advanced_cv_scoresD.mean())

Advanced Regressor CV MSE: 520.5452166182314


# Evaluate

+ Which model has the best performance?

MSE value of Baseline Regressor( LinearRegression) were extremely low compared to the MSE of advanced Regressor( Random Forest Regressor), indicating Basline Regression has the best performance.

# Export

+ Save the best performing model to a pickle file.

In [193]:
print(fires_dt.columns)


Index(['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain', 'area'],
      dtype='object')


In [194]:
import pickle

# Assuming baseline_grid_search is the best performing model based on your previous results
best_model = baseline_grid_searchB.best_estimator_

# Save the model to a pickle file
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Best performing model saved to 'best_model.pkl'.")


Best performing model saved to 'best_model.pkl'.


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap
X = fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']]
y = fires_dt['area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

PipelineA.fit(X_train,y_train)
model = PipelineA.named_steps['regressor']
explainer = shap.Explainer(PipelineA, X_train) 
shap_values = explainer(X_test)  
observation = X_test.iloc[0] 
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], observation)
shap.summary_plot(shap_values, X_test)  

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.